In [74]:
import os
import uuid
from notebookutils import mssparkutils
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, MapType, ArrayType,BooleanType

f_uuid = F.udf(lambda: str(uuid.uuid4()), StringType())
f_bool = F.udf(lambda: True, BooleanType())


StatementMeta(sparkpool01, 8, 1, Finished, Available)

In [75]:
mssparkutils.fs.mount( 
    "abfss://bronze@xxxxxxx.dfs.core.windows.net", 
    "/mydata", 
    {"linkedService":"xxxxxx-synapse-01-WorkspaceDefaultStorage"} 
)
job_id = mssparkutils.env.getJobId()
bronze_mount_point = f"synfs:/{job_id}/mydata/archive.zip"

StatementMeta(sparkpool01, 8, 2, Finished, Available)

In [88]:
cosmosEndpoint = "https://ebcbin5oofjcs.documents.azure.com:443/"
cosmosMasterKey = "xxxxx"
cosmosDatabaseName = "database01"
cosmosContainerName = "graph01"

cfg = {
  "spark.cosmos.accountEndpoint" : cosmosEndpoint,
  "spark.cosmos.accountKey" : cosmosMasterKey,
  "spark.cosmos.database" : cosmosDatabaseName,
  "spark.cosmos.container" : cosmosContainerName,
}
# Configure Catalog Api to be used
spark.conf.set("spark.sql.catalog.cosmosCatalog", "com.azure.cosmos.spark.CosmosCatalog")
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountEndpoint", cosmosEndpoint)
spark.conf.set("spark.sql.catalog.cosmosCatalog.spark.cosmos.accountKey", cosmosMasterKey)

def write_to_cosmos_graph(df: DataFrame, data_type: str, save: bool = False):
    if (save):
        df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(f"synfs:/{job_id}/mydata/{data_type}/")
        
    df.write\
   .format("cosmos.oltp")\
   .options(**cfg)\
   .mode("APPEND")\
   .save()

StatementMeta(sparkpool01, 8, 15, Finished, Available)

In [77]:
raw_data = spark.read.format('delta').load(bronze_mount_point).distinct()

StatementMeta(sparkpool01, 8, 4, Finished, Available)

In [78]:
# Create vertices based on activity and num of transaction made from raw data
def prepare_vertices(df: DataFrame) -> DataFrame:
    nameOrig = df.select("nameOrig")
    nameDest = df.select("nameDest")
    busy_accounts = nameOrig.union(nameDest).withColumnRenamed('nameOrig','id')
    top_account_activity = busy_accounts.groupBy('id').count().filter(F.col('count') > 10 ).select('id') # accounts with more than 10 transactions
    top_account_amount = df.filter(F.col('amount') > 500000) # accounts with amount transfer more than 500000
    nameOrigA = top_account_amount.select('nameOrig')
    nameDestA = top_account_amount.select('nameDest')
    top_account_transact = nameOrigA.union(nameDestA).withColumnRenamed('nameOrig','id').distinct().select('id')
    vertices = top_account_activity.union(top_account_transact).distinct()
    cosmos_vertices_df = vertices.withColumn('label',F.lit('account')).withColumn('accountId',vertices['id']).\
    select("label","id","accountId")
    return cosmos_vertices_df.distinct()


StatementMeta(sparkpool01, 8, 5, Finished, Available)

In [79]:
def prep_cosmos_edge_df(df: DataFrame) -> DataFrame:
   return df.distinct().withColumn("label",df['type']).\
    withColumn('accountId',df['nameOrig']).\
    withColumnRenamed('nameOrig','_vertexId').\
    withColumn('_sinkPartition',df['nameDest']).\
    withColumnRenamed('nameDest',"_sink").\
    withColumn('_sinkLabel',F.lit("account")).\
    withColumn('_vertexLabel',F.lit("account")).\
    withColumn('_isEdge',f_bool()).\
    withColumn('id', f_uuid()).\
    select("id","label","_sink","_sinkLabel","_sinkPartition","_vertexId","_vertexLabel","_isEdge","accountId","type","amount","oldbalanceOrg","oldbalanceDest","newbalanceDest")


# Filter transaction that have either the source or destination as vertices
def prepare_edges_loose(df: DataFrame, vertices:DataFrame) -> DataFrame:
    transactions_nameOrig = df.join(vertices, df['nameOrig'] == vertices['id'])
    transactions_nameDest = df.join(vertices, df['nameDest'] == vertices['id'])
    edges = transactions_nameOrig.union(transactions_nameDest).select("type","amount","nameOrig","nameDest","oldbalanceOrg","newbalanceOrig","oldbalanceDest","newbalanceDest")
    return prep_cosmos_edge_df(edges)
    
    

# Filter transaction that have both source and destination as vertices
def prepare_edges_strict(df: DataFrame, vertices:DataFrame) -> DataFrame:
    transactions = df.alias("a").join(
        vertices.alias("b"), df.nameOrig == vertices.id)\
        .drop( vertices.id)\
        .join(
            vertices.alias("c"), df.nameDest == vertices.id)
    edges = transactions.select("type","amount","nameOrig","nameDest","oldbalanceOrg","newbalanceOrig","oldbalanceDest","newbalanceDest")
    return prep_cosmos_edge_df(edges)



StatementMeta(sparkpool01, 8, 6, Finished, Available)

In [80]:
v = prepare_vertices(raw_data)

StatementMeta(sparkpool01, 8, 7, Finished, Available)

In [60]:
v.count()

StatementMeta(sparkpool01, 7, 45, Finished, Available)

505821

In [81]:
e = prepare_edges_strict(raw_data,v)

StatementMeta(sparkpool01, 8, 8, Finished, Available)

In [71]:
e.show()

StatementMeta(sparkpool01, 7, 56, Finished, Available)

+--------------------+--------+-----------+----------+--------------+-----------+------------+-------+-----------+--------+----------+-------------+--------------+--------------+
|                  id|   label|      _sink|_sinkLabel|_sinkPartition|  _vertexId|_vertexLabel|_isEdge|  accountId|    type|    amount|oldbalanceOrg|oldbalanceDest|newbalanceDest|
+--------------------+--------+-----------+----------+--------------+-----------+------------+-------+-----------+--------+----------+-------------+--------------+--------------+
|f28235ac-a9ba-4c4...|TRANSFER|C1002140910|   account|   C1002140910|C1348263973|     account|   true|C1348263973|TRANSFER| 543109.13|          0.0|    1479995.34|    2023104.48|
|189c0c71-777b-45f...|CASH_OUT|C1002735434|   account|   C1002735434| C540661761|     account|   true| C540661761|CASH_OUT| 516655.06|      10040.0|     395505.22|     912160.27|
|452b63bb-cb3b-45d...|TRANSFER|C1003597854|   account|   C1003597854|C1077134277|     account|   true|C10

In [72]:
e.count()

StatementMeta(sparkpool01, 7, 57, Finished, Available)

340762

In [9]:
# # Create test data set to verify vertices and edges are being created correctly
# cosmos_edges_df_test = cosmos_edges_df.limit(10)
# source = cosmos_edges_df_test.select('_vertexId')
# dest = cosmos_edges_df_test.select('_sink')
# tmp = source.union(dest).distinct().withColumnRenamed('_vertexId','id').withColumn('label',F.lit('account'))
# cosmos_vertices_df_test = tmp.withColumn('accountId',tmp['id']).select("label","id","accountId")
# print(f"Number of Accounts/Vertices: {cosmos_vertices_df_test.count()}, Number of transactions/Edges: {cosmos_edges_df_test.count()}")
# write_to_cosmos_graph(cosmos_vertices_df_test,"vertices",False)
# write_to_cosmos_graph(cosmos_edges_df."edges",False)

StatementMeta(sparkpool01, 6, 9, Finished, Available)

Number of Accounts/Vertices: 20, Number of transactions/Edges: 10

In [89]:
write_to_cosmos_graph(v,"vertices",True)
write_to_cosmos_graph(e,"edges",True)

StatementMeta(sparkpool01, 8, 16, Finished, Available)

In [90]:
edge_mount = f"synfs:/{job_id}/mydata/edges"
vertices_mount = f"synfs:/{job_id}/mydata/vertices"

StatementMeta(, , , Finished, )

: Livy session has failed. Session state: Killed. Session has been cancelled. Source: User.

In [29]:
loaded_edges = spark.read.load(edge_mount)
loaded_edges.show()

StatementMeta(sparkpool01, 7, 14, Finished, Available)

+--------+--------------------+-----------+
|   label|                  id|  accountId|
+--------+--------------------+-----------+
|CASH_OUT|2e9c9334-10ba-49a...| C607662432|
| CASH_IN|99c809cf-e00b-4a5...| C298828698|
|CASH_OUT|9b9c9c8c-1e2e-4e1...|C2009902265|
| CASH_IN|a0464aca-093c-452...| C927021051|
|CASH_OUT|a428a808-a8e2-487...|C1389813286|
|CASH_OUT|b024bfa2-d21c-4b6...| C254459874|
|CASH_OUT|5a5fc2b2-d7e6-4db...| C478026954|
|CASH_OUT|4562d693-73bd-445...|C1940951344|
|CASH_OUT|c71b9272-f9b2-4ff...|C1191586887|
| CASH_IN|19d2cd86-730b-4a8...| C349558668|
| CASH_IN|f418d8c7-206c-490...|C1723926446|
| CASH_IN|4ca4ed41-340b-4a5...|C1617085732|
| CASH_IN|0a5f085f-4491-403...|C1152435068|
|CASH_OUT|19a05e34-a933-429...|C1760052444|
|CASH_OUT|b44133e6-013f-471...| C630550318|
|CASH_OUT|05a987d8-b542-415...|C1165673445|
| CASH_IN|6066dd60-a45a-493...| C278443878|
|CASH_OUT|3073381f-7d77-4b6...| C487967778|
|TRANSFER|205f5490-81f2-467...|C1897482719|
| CASH_IN|7ca5c52d-fe6c-43e...| 

In [31]:
vertices = spark.read.load(vertices_mount)
vertices.show()

StatementMeta(sparkpool01, 7, 16, Finished, Available)

+-------+-----------+-----------+
|  label|         id|  accountId|
+-------+-----------+-----------+
|account| C771213975| C771213975|
|account| C286977404| C286977404|
|account|C1994971777|C1994971777|
|account|C1552859894|C1552859894|
|account|C1376626882|C1376626882|
|account|C1499489682|C1499489682|
|account| C283471991| C283471991|
|account| C251453626| C251453626|
|account| C407287420| C407287420|
|account| C469767556| C469767556|
|account| C542245023| C542245023|
|account| C614274623| C614274623|
|account|C1081632645|C1081632645|
|account| C361050839| C361050839|
|account|C1818824163|C1818824163|
|account|C1328577807|C1328577807|
|account| C177570699| C177570699|
|account|C1059072014|C1059072014|
|account| C339119211| C339119211|
|account|C1935730862|C1935730862|
+-------+-----------+-----------+
only showing top 20 rows